STEP 1 : Data scrapping from kayak

In [1]:
import requests as rq
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from time import sleep
import lxml
from lxml import html

In [4]:
driver = webdriver.Edge()

In [5]:
#testing if the driver is working correctly
source='CMN'
destination='PAR'
date='2024-09-09'
kayak=f"https://www.kayak.ae/flights/{source}-{destination}/{date}/2024-09-09?sort=bestflight_a"
driver.get(kayak)
soup = BeautifulSoup(driver.page_source, 'html.parser')
all_flights=soup.find_all('div',class_='nrc6-content-section')
#get airlines
airlines=[]
for flight_det in all_flights :
        item=flight_det.find_all('li',class_='hJSA-item')
        img=item[0].find('img',class_='')
        alt_text = img.get('alt', '')
        airlines.append(alt_text)

airlines

['Royal Air Maroc',
 'TAP Portugal',
 'Royal Air Maroc',
 'Royal Air Maroc',
 'Royal Air Maroc',
 'Royal Air Maroc',
 'Royal Air Maroc',
 'Air France',
 'Air France',
 'Royal Air Maroc',
 'Air France',
 'Royal Air Maroc',
 'Royal Air Maroc',
 'Royal Air Maroc',
 'Royal Air Maroc',
 'Air France']

NOW WE WILL MOVE ON TO COLLECTING THE DATA WE NEED

In [6]:
def click_show_more(driver):
    try:
        show_more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="c8hZ6"]/div/div'))
        )

        # Scroll to the "Show More" button to make it clickable
        ActionChains(driver).move_to_element(show_more_button).perform()
        time.sleep(1)  # Add a small delay to ensure the button is clickable

        # Click the "Show More" button
        show_more_button.click()

        # You may need to wait for the new content to load before proceeding
        WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.XPATH, 'loading_indicator_xpath')))
        
    except Exception as e:
        print(f"An error occurred while clicking 'Show More': {e}")



In [7]:
#you specify the sources destinations and dates 

sources=['CMN','PAR','LAX','IST','NRT']
destinations=['CMN','PAR','LAX','IST','NRT']
classes=['economy','business']
Dates_1 = [
    '2024-05-29', '2024-05-30', '2024-05-31', '2024-06-01', '2024-06-02',
    '2024-06-03', '2024-06-04', '2024-06-05', '2024-06-06', '2024-06-07',
    '2024-06-08', '2024-06-09', '2024-06-10', '2024-06-11', '2024-06-12',
    '2024-06-13', '2024-06-14', '2024-06-15', '2024-06-16', '2024-06-17',
    '2024-06-18', '2024-06-19', '2024-06-20', '2024-06-21', '2024-06-22',
    '2024-06-23', '2024-06-24', '2024-06-25', '2024-06-26', '2024-06-27',
    '2024-06-28', '2024-06-29', '2024-06-30', '2024-07-01', '2024-07-02',
    '2024-07-03', '2024-07-04', '2024-07-05', '2024-07-06', '2024-07-07',
    '2024-07-08'
]

testdate=['2024-07-08']


In [8]:
def get_airline(soup):
    #get airlines
    airlines=[]
    items=soup.find_all('div',class_='J0g6-operator-text') 
    for item in items :
        airl=item.text
        airl = airl.split(',')[0].strip()
        airlines.append(airl)
    
    return airlines


In [9]:
def get_duration(soup):
    #getduration
    durations=[]
    duration_div=soup.find_all('div',class_='xdW8 xdW8-mod-full-airport')
    for d in duration_div :
        duration=d.find('div',class_='vmXl vmXl-mod-variant-default').text
        durations.append(duration)
    return durations

In [10]:
def get_stops(soup):
    stops=[]
    stop_div=soup.find_all('div',class_='vmXl vmXl-mod-variant-default')
    for s in stop_div :
        selected_element = s.select_one('.JWEO-stops-text, .JWEO-mod-warning')
        if selected_element:
          stops.append(selected_element.text)

    return stops

In [11]:
def get_price(soup):   
    #getprice
    prices=[]
    price_div=soup.find_all('div',class_='f8F1-price-text-container')

    for p in price_div :
        price=p.find('div',class_='f8F1-price-text').text
        price= price.replace('\xa0', ' ')
        prices.append(price)
    return prices

In [12]:
def get_class(soup):
    #getclass
    classes=[]
    class_div=soup.find_all('div',class_='aC3z-option')
    for c in class_div :
        classe=c.find('div',class_='aC3z-name').text
        classes.append(classe)
    return classes

In [13]:
#get_depature_time
def get_dep_time(soup):
    deps=[]
    dep_div=soup.find_all('div',class_='vmXl vmXl-mod-variant-large')
    for d in dep_div:
        depature=d.find_all('span')
        deps.append(depature[0].text)
    return deps
#get_arrival_time
def get_arr_time(soup):
    arrs=[]
    arr_div=soup.find_all('div',class_='vmXl vmXl-mod-variant-large')
    for a in arr_div:
        arrival=a.find_all('span')
        arrs.append(arrival[2].text)
    return arrs

In [14]:
df=pd.DataFrame(columns=["Airline", "Source", "Destination","Duration","stops","class","depature time","arrival time", "Price","Date"])
for k in range(len(testdate)): #replace testdate with dates_1 that is a list of our wanted dates
    for j in range(len(classes)):    
        for i in range(len(sources)): 
            for l in range(len(destinations)):
                if sources[i]==destinations[l]:
                    continue
                driver = webdriver.Edge()
                kayak=f"https://www.kayak.ae/flights/{sources[i]}-{destinations[l]}/{testdate[k]}/{classes[j]}?sort=bestflight_a"
                print(Dates_1[k])
                print(sources[i])
                print(destinations[l])
                print(classes[j])
                driver.get(kayak)
                # Click "Show More" button
                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'ULvh-button.show-more-button')))
                '''count=2
                while count>1:
                    try:
                        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'ULvh-button.show-more-button'))).click()
                        print("MORE button clicked")
                    except TimeoutException:
                        break
                    count-=1
                    sleep(5)'''

                soup = BeautifulSoup(driver.page_source, 'html.parser')
                airlines = get_airline(soup)
                print(len(airlines))
                stops = get_stops(soup)
                print(len(stops))
                dep_time=get_dep_time(soup)
                print(len(dep_time))
                arr_time =get_arr_time(soup)
                print(len(arr_time))
                prices = get_price(soup)
                print(len(prices))
                duration = get_duration(soup)
                print(len(duration))
                df = pd.concat([df,pd.DataFrame({
                                'Airline': airlines,
                                'Duration': duration,
                                'stops' : stops,
                                'class' : classes[j],
                                'Source':sources[i],
                                'Destination': destinations[l],
                                'depature time' : dep_time ,
                                'arrival time' : arr_time,
                                'Price' : prices,
                                'Date' : Dates_1[k]})])


                df = df.replace('\n','', regex=True)
                #df = df.reset_index(drop = True)
                # save data as csv file for each route

                print(f"Succesfully saved {sources[i]} => {destinations[l]} route  ")
                driver.quit()
#df.to_csv('../data/airlines_dataset.csv', index=False, mode='a', header=False)
#uncomment this when you want to save the data scrapped after you test the code with testdate
driver.quit()


2024-05-29
CMN
PAR
economy
16
16
16
16
16
16
Succesfully saved CMN => PAR route  
2024-05-29
CMN
LAX
economy
15
15
15
15
15
15
Succesfully saved CMN => LAX route  
2024-05-29
CMN
IST
economy
16
16
16
16
16
16
Succesfully saved CMN => IST route  
2024-05-29
CMN
NRT
economy
16
16
16
16
16
16
Succesfully saved CMN => NRT route  
2024-05-29
PAR
CMN
economy
16
16
16
16
16
16
Succesfully saved PAR => CMN route  
2024-05-29
PAR
LAX
economy
15
15
15
15
15
15
Succesfully saved PAR => LAX route  
2024-05-29
PAR
IST
economy
16
16
16
16
16
16
Succesfully saved PAR => IST route  
2024-05-29
PAR
NRT
economy
16
16
16
16
16
16
Succesfully saved PAR => NRT route  
2024-05-29
LAX
CMN
economy
16
16
16
16
16
16
Succesfully saved LAX => CMN route  
2024-05-29
LAX
PAR
economy
15
15
15
15
15
15
Succesfully saved LAX => PAR route  
2024-05-29
LAX
IST
economy
16
16
16
16
16
16
Succesfully saved LAX => IST route  
2024-05-29
LAX
NRT
economy
15
15
15
15
15
15
Succesfully saved LAX => NRT route  
2024-05-29
IST
C

In [16]:
df

,Airline,Source,Destination,Duration,stops,class,depature time,arrival time,Price,Date
0,Transavia France,CMN,PAR,3h 00m,direct,economy,17:45,21:45,AED 282,2024-05-29
1,TAP Portugal,CMN,PAR,25h 25m,1 stop,economy,11:25,13:50+1,AED 254,2024-05-29
2,Royal Air Maroc,CMN,PAR,2h 50m,direct,economy,16:50,20:40,AED 522,2024-05-29
3,Royal Air Maroc,CMN,PAR,2h 50m,direct,economy,13:00,16:50,AED 522,2024-05-29
4,Royal Air Maroc,CMN,PAR,2h 55m,direct,economy,13:25,17:20,AED 522,2024-05-29
...,...,...,...,...,...,...,...,...,...,...
10,Etihad Airways,NRT,IST,23h 30m,2 stops,business,19:50,13:20+1,"AED 9,415",2024-05-29
11,Ethiopian Air,NRT,IST,29h 55m,2 stops,business,21:15,21:10+1,"AED 8,220",2024-05-29
12,Etihad Airways,NRT,IST,29h 20m,2 stops,business,14:00,13:20+1,"AED 9,415",2024-05-29
13,Etihad Airways,NRT,IST,29h 20m,2 stops,business,14:00,13:20+1,"AED 9,415",2024-05-29
